In [ ]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import re
from copy import deepcopy
from collections import Counter

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
from nltk.tokenize import TreebankWordTokenizer

In [ ]:
from nltk.tag import pos_tag

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.decomposition import TruncatedSVD

from scipy.sparse.linalg import svds
from scipy.sparse import coo_matrix

from sklearn.cluster import KMeans

**ORIGINAL DATA SET CAN BE FOUND HERE:**

[Kaggle Fake News Dataset](https://www.kaggle.com/mrisdal/fake-news/downloads/fake-news.zip)

In [ ]:
zipfile_path = '/home/farhaan/Downloads/fake-news.zip'
df_fakenews = pd.read_csv(zipfile_path,compression="zip")

In [ ]:
df_fakenews.head(3)

In [ ]:
list(df_fakenews.loc[:,'text'].sample(1))

In [ ]:
df_fakenews.shape

In [ ]:
df_fakenews['type'].value_counts()

In [ ]:
str_lengths = df_fakenews.text.str.len()
len(str_lengths[str_lengths>1000])

In [ ]:
plt.hist(str_lengths[str_lengths>1000],bins=40);

## Filtered fakenews dataframe

In [ ]:
fakenews_df_filtered = df_fakenews[pd.notnull(df_fakenews['text'])]
fakenews_df_filtered = fakenews_df_filtered[fakenews_df_filtered['language']=='english']
delregex1= re.compile('(?:Be the First to Comment.*articles *$|Subscribe *$|\nRelated:.*$| Share:.*$)',re.IGNORECASE|re.DOTALL)
delregex2 = re.compile('(?:Submit your review|Follow on (?:Facebook|Twitter)|Print this post|Share on (?:Facebook|Twitter))',re.IGNORECASE)
delregex3 = re.compile('(?:Century Wire|Newsinfo|Delivered by The Daily Sheeple\nWe .*$|barracuda brigade)',re.IGNORECASE)
delregex4 = re.compile('READ MORE ELECTION NEWS AT .*$',re.IGNORECASE,re.DOTALL)
delregex5 = re.compile('^.{0,300}Leave a Comment |Leave a Reply Click here to get more.*',re.IGNORECASE|re.DOTALL)
delregex6 = re.compile('(?:SUBSCRIBE on YouTube|[\n\.].*?like and share .*|^Share on Twitter.*\.|This article was written by .*?\.)',re.IGNORECASE)
# delregex7 = re.compile('\nRelated:.*$',re.IGNORECASE|re.DOTALL)
# delregex8 = re.compile('Subscribe *$',re.IGNORECASE)
for ind,txt in zip(fakenews_df_filtered.index,fakenews_df_filtered.text):
    txt = re.sub(delregex1,"",txt)
    txt = re.sub(delregex2,"",txt)
    txt = re.sub(delregex3,"",txt)
    txt = re.sub(delregex4,"",txt)
    txt = re.sub(delregex5,"",txt)
    txt = re.sub(delregex6,"",txt)
    

#     txt = re.sub(delregex7,"",txt)
#     txt = re.sub(delregex8,"",txt)
    fakenews_df_filtered.set_value(ind,'text',txt)
fakenews_df_filtered = fakenews_df_filtered[fakenews_df_filtered.text.str.len()>850]
text= fakenews_df_filtered.text

In [ ]:
len(fakenews_df_filtered)

In [ ]:
state = 30
index1 = fakenews_df_filtered.sample(random_state=state).index
print(list(fakenews_df_filtered.sample(random_state=state).index),'\n',list(fakenews_df_filtered.sample(random_state=state).text))

In [ ]:
list(df_fakenews.iloc[index1,5])

## Count vectorizer and PCA, straight up 

In [ ]:
vectorizer = CountVectorizer(strip_accents='ascii',analyzer='word',lowercase=True,stop_words='english',min_df=10)

In [ ]:
vectorizer.fit(text)

In [ ]:
# print(vectorizer.get_feature_names())

In [ ]:
len(vectorizer.get_feature_names())

In [ ]:
feat_names_dirty = vectorizer.get_feature_names()

In [ ]:
regex='[^\d_]*'
feat_names_clean= [re.findall(regex,s)[0] for s in feat_names_dirty if (re.findall(regex,s)!=[] and len(re.findall(regex,s)[0])>=4 and len(re.findall(regex,s)[0])<20)]

feat_names_clean= list(set(feat_names_clean))

In [ ]:
# [w for w in feat_names_dirty if len(w)>=20]

In [ ]:
Counter([len(i) for i in feat_names_clean])

In [ ]:
# print(feat_names_clean)

In [ ]:
len(feat_names_clean)

In [ ]:
vectorizer = CountVectorizer(vocabulary=feat_names_clean)

In [ ]:
vectorizer.fit(text)

In [ ]:
print(vectorizer.get_feature_names())

In [ ]:
wd_frequency = vectorizer.transform(text)

In [ ]:
u,s,vt = svds(wd_frequency.asfptype(),k=200)

In [ ]:
u.shape

In [ ]:
s

In [ ]:
cumul=0
n_components=[]
cum_vals = []
for ind,singv in enumerate(sorted(s,reverse=True)):
    cumul += singv
    n_components.append(ind)
    cum_vals.append(cumul/sum(s))

In [ ]:
plt.plot(n_components,cum_vals,'g-')

In [ ]:
tsvd = TruncatedSVD(n_components=300,random_state=40)

In [ ]:
tsvd.fit(wd_frequency)

In [ ]:
tsvd.get_params()

In [ ]:
sum(tsvd.explained_variance_ratio_)

In [ ]:


cumul=0
n_components=[]
cum_vals = []
for ind,singv in enumerate(tsvd.explained_variance_ratio_):
    cumul += singv
    n_components.append(ind+1)
    cum_vals.append(cumul)
    
plt.plot(n_components,cum_vals,'g-');

In [ ]:
transformed_wd_frequency = pd.DataFrame(tsvd.transform(wd_frequency))

In [ ]:
kmeanscl = KMeans(n_clusters=10,n_jobs=-1,random_state=40)

kmeanscl.fit(transformed_wd_frequency)

centroids = kmeanscl.cluster_centers_

Counter(kmeanscl.labels_)

In [ ]:
len(kmeanscl.labels_)

In [ ]:
clustered_text = pd.merge(pd.DataFrame(fakenews_df_filtered),
                          pd.DataFrame(kmeanscl.labels_,
                                       index=fakenews_df_filtered.index,
                                       columns=['cluster_label']),
                          how='left',left_index=True,right_index=True)

In [ ]:
list(clustered_text[clustered_text['cluster_label']==4]['text'].sample())

In [ ]:
clustered_text[clustered_text['language']!='english']

In [ ]:
clustered_text[clustered_text['cluster_label']==8]

In [ ]:
[list(print(doc,'\n\n') for doc in clustered_text[clustered_text['cluster_label']==2].sample(5).text)]

## TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(strip_accents='ascii',analyzer='word',lowercase=True,stop_words='english',min_df=10,vocabulary=feat_names_clean)

In [ ]:
vectorizer.fit(text)

In [ ]:
len(vectorizer.get_feature_names())

In [ ]:
wd_frequency = vectorizer.transform(text)
tsvd = TruncatedSVD(n_components=300,random_state=40)
tsvd.fit(wd_frequency)

In [ ]:
cumul=0
n_components=[]
cum_vals = []
for ind,singv in enumerate(tsvd.explained_variance_ratio_):
    cumul += singv
    n_components.append(ind+1)
    cum_vals.append(cumul)
    
plt.plot(n_components,cum_vals,'g-');

In [ ]:
sorted([(ind,val) for ind,val in enumerate(tsvd.components_[0])],key=lambda x: x[1],reverse=True)[:10]

In [ ]:
test_indices = [10187,10824,2465,10654,3326,11731]
[feat_names_clean[i] for i in test_indices]

In [ ]:
transformed_wd_frequency = pd.DataFrame(tsvd.transform(wd_frequency))

In [ ]:
kmeanscl = KMeans(n_clusters=10,n_jobs=-1,random_state=40)

kmeanscl.fit(transformed_wd_frequency)

centroids = kmeanscl.cluster_centers_

Counter(kmeanscl.labels_)

In [ ]:
sorted([(ind,val) for ind,val in enumerate(centroids[9])],key=lambda x: x[1],reverse=True)

In [ ]:
centroids[0]

In [ ]:
# to get the most important topics in the cluster:
for cluster_ind in range(len(centroids)):
    main_topics_in_cluster = [topic[0] for topic in sorted([(ind,val) for ind,val in enumerate(centroids[cluster_ind])],key=lambda x: x[1],reverse=True)[:3]]
    topic_relative_importance = [topic[1] for topic in sorted([(ind,val) for ind,val in enumerate(centroids[cluster_ind])],key=lambda x: x[1],reverse=True)[:3]]
    print(f"Cluster # {cluster_ind}",main_topics_in_cluster)
    print(f"Topic rel. importance: {np.around(topic_relative_importance,3)}\n")
    for topic in main_topics_in_cluster:
        words_in_topic = [word_ind[0] for word_ind in sorted([(ind,val) for ind,val in enumerate(tsvd.components_[topic])],key=lambda x: x[1],reverse=True)[:15]]
        print(f"Topic # {topic}",[feat_names_clean[word_ind] for word_ind in words_in_topic])
    excerpts = [clustered_text[clustered_text['cluster_label']==cluster_ind].loc[:,'text'].iloc[i][200:600] for i in range(5,8)]
    for ind,excerpt in enumerate(excerpts):
        print(f"\n*******Excerpt#{ind+1}: *********\n {excerpt}")
    print('\n\n')
    print("___________________________________________________________________________")

In [ ]:
clustered_text = pd.merge(pd.DataFrame(fakenews_df_filtered),
                          pd.DataFrame(kmeanscl.labels_,
                                       index=fakenews_df_filtered.index,
                                       columns=['cluster_label']),
                          how='left',left_index=True,right_index=True)

In [ ]:
list(clustered_text[clustered_text['cluster_label']==9].loc[:,'text'].iloc[i][300:1000] for i in range(10))

# SPACY

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en')

In [ ]:
mod_text = deepcopy(text)

In [ ]:
cleanText(mod_text[1])

In [ ]:
mod_docs = [doc for doc in nlp.pipe(mod_text,n_threads=3)]

In [ ]:
doc_idxs = [doc_idx for doc_idx in text.index]

In [ ]:
documents = [nlp(doc) for doc in text]

In [ ]:
text

In [ ]:
ttxt = clustered_text[clustered_text['cluster_label']==4].text[10337]

In [ ]:
ttxt

In [ ]:
documents[14]

In [ ]:
for token in documents[3]:
    print(token.lemma_)

In [ ]:

stw = list(stopwords.words('english'))
stw +=['.',',','!','?','"',"'"]

In [ ]:
string.punctuation

In [ ]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string

In [ ]:
# A custom stoplist
content_specific_stopwords = ['trump', 'clinton', 'hillary', 'election', 'people', 'said', 'president', 'obama', 'campaign']
STOPLIST = set(stopwords.words('english') + ["n't", "'s", "'m", "ca","’s","'ve"] + list(ENGLISH_STOP_WORDS)+content_specific_stopwords)
# List of symbols we don't care about
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---",'--','-', "...", "“", "”",'…' ,'’','–']

# Every step in a pipeline needs to be a "transformer". 
# Define a custom transformer to clean text using spaCy
class CleanTextTransformer(TransformerMixin):
    """
    Convert text to cleaned text
    """

    def transform(self, X, **transform_params):
        return [cleanText(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}
    
# A custom function to clean the text before sending it into the vectorizer
def cleanText(text):
    
    # get rid of newlines
    text = text.strip().replace("\n", " ").replace("\r", " ")
    
    # split up words comlicated by some symbols
    text = text.replace('’'," ")
    
    #convert text to unicode
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) # no emoji
    
    # replace twitter @mentions
    mentionFinder = re.compile(r"@[a-z0-9_]{1,15}", re.IGNORECASE)
    text = mentionFinder.sub("", text)
    
    # replace HTML symbols
    bracs_regex = re.compile('<.*?>')
    text = re.sub(bracs_regex," ",text)
    text = text.replace("&amp;", "and").replace("&gt;", ">").replace("&lt;", "<")
    
    # delete urls
    url_reg = re.compile('\w+(?:\.\w+)+')
    text = re.sub(url_reg, "", text)
    url_reg2 = re.compile('http.* ')
    text = re.sub(url_reg2, " ", text)
    
    # delete words countaining numbers
    num_reg = re.compile('\w*\d+\w*')
    text = re.sub(num_reg, "", text)
        
    # lowercase
    text = text.lower()
    
    # only return alphabetical words
    letter_reg = re.compile(' .*[^a-z]+?.* ')
    text = re.sub(letter_reg,' ',text)

    return text

# A custom function to tokenize the text using spaCy
# and convert to lemmas
def tokenizeText(sample):

    # get the tokens using spaCy
    tokens = nlp(sample)

    # lemmatize
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas

    # stoplist the tokens
    tokens = [tok for tok in tokens if tok not in STOPLIST]

    # stoplist symbols
    tokens = [tok for tok in tokens if tok not in SYMBOLS]

    # remove large strings of whitespace
    while "" in tokens:
        tokens.remove("")
    while " " in tokens:
        tokens.remove(" ")
    while "\n" in tokens:
        tokens.remove("\n")
    while "\n\n" in tokens:
        tokens.remove("\n\n")

    tokens = [token for token in tokens if len(token)>=3]
    return tokens

def printNMostInformative(vectorizer, clf, N):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    topClass1 = coefs_with_fns[:N]
    topClass2 = coefs_with_fns[:-(N + 1):-1]
    print("Class 1 best: ")
    for feat in topClass1:
        print(feat)
    print("Class 2 best: ")
    for feat in topClass2:
        print(feat)
    
# the vectorizer and classifer to use
# note that I changed the tokenizer in CountVectorizer to use a custom function using spaCy's tokenizer
vectorizer = TfidfVectorizer(tokenizer=tokenizeText, ngram_range=(1,1))
clf =  KMeans(n_clusters=10,n_jobs=-1,random_state=40)
# the pipeline to clean, tokenize, vectorize, and classify
pipe = Pipeline([('cleanText', CleanTextTransformer()), ('vectorizer', vectorizer), ('clf', clf)])

In [ ]:
cl_text_tf = CleanTextTransformer()

In [ ]:
cl_text_tf.fit(mod_text)

In [ ]:
cl_txt = cl_text_tf.transform(mod_text)

In [ ]:
cl_txt[0]

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=tokenizeText, ngram_range=(1,1))

In [ ]:
vectorizer.fit(cl_txt)

In [ ]:
sparsed_txt = vectorizer.transform(cl_txt)

In [ ]:
vectorizer.get_feature_names()

In [ ]:
type(sparsed_txt)

In [ ]:
tsvd = TruncatedSVD(n_components=80,random_state=40)
tsvd.fit(sparsed_txt)

In [ ]:
sum(sorted(tsvd.explained_variance_ratio_))

In [ ]:
kmeanscl = KMeans(n_clusters=8,n_jobs=-1,random_state=40)

transformed_wd_frequency = pd.DataFrame(tsvd.transform(sparsed_txt))

kmeanscl.fit(transformed_wd_frequency)

centroids = kmeanscl.cluster_centers_

Counter(kmeanscl.labels_)

In [ ]:
# data
train = mod_text

# train
pipe.fit(train)

In [ ]:
for sent in list(documents[0].sents)[:10]:
    for token in sent:
        print(token.lemma_)

In [ ]:
len(text)

In [ ]:
from spacy.en import English
parser = English()

In [ ]:
testtoklist = cleanText(text[5])

In [ ]:
type(testtoklist[0])

In [ ]:
tst2=deepcopy(text[5])

In [ ]:
regex = re.compile('\w+(?:\.\w+)+')

In [ ]:
re.findall(regex,tst2)

In [ ]:
list(ENGLISH_STOP_WORDS)

In [ ]:
fakenews_df_filtered

In [ ]:
mod_text = deepcopy(text)

In [ ]:
import gensim

In [ ]:
from gensim import corpora, models, similarities

In [ ]:
tfidf = models.TfidfModel(documents[0])